# PROYECTO III  PROGRAMA DE FORMACIÓN  MLDS AVANZADO
## Daniel F. Benavides R. 
## Módulo VI - Entrenamiento de modelo de red neuronal y disposición del mismo a nivel local. 

### OBJETIVO

El objetivo de este proyecto es realizar el despliegue  de un modelo a nivel local. El mismo se llevará a cabo en dos partes: La primera en la cual se realiza el entrenamiento del modelo. El mismo se guarda a nivel local para su posterior uso. 

Es así como a continuación se ve el ejercicio de fine-tuning del modelo preentrenado de transformers  [_'distilbert-base-uncased'_](https://huggingface.co/distilbert-base-uncased?text=Paris+is+the+%5BMASK%5D+of+France.). Este modelo inicialmente fue entrenado para labores de _fill mask_ y se adaptará  como modelo clasificación  de **SMS** no  deseado. 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Carga y manipulación de los datos 

A continuación importamos pandas, por medio del cual hacemos el respectivo cargue del dataset, delimitamos por el espacio la  etiqueta del mensaje.

Luego por medio de la función _list_ convertimos el mensaje y las etiquetas en un par de listas. luego convertimos las etiquetas en una variable dummie, debido a que tenemos una salida binaria  _(el mensaje es spam o no lo es)_

## Importamos el dataset de entrenamiento

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path= "/content/drive/MyDrive/MLDS-2/MODULO II/Talleres/SMSSpamCollection"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

import pandas as pd
df=messages = pd.read_csv(path, sep='\t',
                           names=["label", "message"])
X=list(df['message'])
y=list(df['label'])
y=list(pd.get_dummies(y,drop_first=True)['spam'])


### Preprocesamiento 

Ahora importamos la función *train_test_split*  del módulo *model_selection* de la librería *scikit-learn*  y por medio de este dividimos  en set de entrenamiento y prueba. Definimos el tamaño de set de prueba en 20% de la muestra. También definimos el parámetro *random_state* para efectos de controlar la generación  de los dos conjuntos de tal manera que  no sean aleatorios. 

Luego instalamos la librería transformers, aunque en mi caso ya lo había realizado. 


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 21.3 MB/s 
     |████████████████████████████████| 182 kB 74.4 MB/s 
     |████████████████████████████████| 7.6 MB 59.5 MB/s 


Ahora debemos invocar los modelos de que vamos a utilizar de la librería transformers  en los siguientes pasos: 

* Llamamos el modelo preentrenado
* Llamamos el tokenizador 

Necesitamos aplicar el tokenizador sobre nuestro conjunto de datos. 

Así que acontinuación llamamos de la librería transformers el tokenizador _"DistilBertTokenizerFast"_ luego lo definimos como nuestro **tokenizer** indicando que el mismo proviene del modelo preentrenado [_'distilbert-base-uncased'_](https://huggingface.co/distilbert-base-uncased?text=Paris+is+the+%5BMASK%5D+of+France.)

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Luego  aplicamos el tokenizador que acabamos de definir sobre nuestro conjunto  de mensajes de entrenamiento y prueba. Como los SMS no tienen la misma longitud (cantidad de tokens) debemos definir los parámetros truncation y padding como True para que se obtener oraciones del mismo tamaño; uno se encarga de rellenar de ceros (padding) y el otro de truncar las oraciones más largas. Esto para obtener un conjunto y luego tensores rectangulares. 

In [ ]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)


Ahora se procede a importar Tensorflow para efecto de convertir en tensores los encodings generados en el paso anterior. Acá se junta cada uno a su correspondiente etiqueta. 

In [ ]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

### Entrenamiento

A continuación se importan los módulos de TFDistilBertForSequenceClassification que es usado para la tarea de clasificación  de sentimientos. También se importan los módulos  y *TFTrainingArguments* y *TFTrainer*que son los encargados de definir los argumentos  y posteriormente parametrizar el **trainer** del modelo y  hacer las nuevas predicciones. 


In [ ]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    eval_steps = 10, 
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

Hemos determinado el conjunto de argumentos que serán utilizados en el reentrenamiento del modelo,estos quedan alojados en  el  objeto *training_args*   y  ahora definiremos el modelo refiriendo el modelo preentrenado que vamos a utilizar, que en este caso es  _"distilbert-base-uncased"_. se creará el **trainer** al cual se le pasarán los argumentos antes definidos   y los dos tensores de entrenamiento y prueba; para luego  entrenar el modelo que hemos definido.

In [ ]:
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)


Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'vocab_projector', 'activation_13', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

una vez instanciado el modelo que será reentrenado,  parametrizados los argumentos para ello, se toma la data y se realiza el reentrenamiento del modelo. 

In [ ]:
trainer.train()

Ahora solo queda por aplicar modelo que reentrenamos con el dataset de **entrenamiento**, hacer la predicción, y la evaluación de las predicciones. Este procedimiento se encuentra definido en el [manual de fine-tuning](https://huggingface.co/transformers/v3.5.1/training.html) que tiene Hugging Face disponible. 

In [ ]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.023609844275883266}

A continuación aplicamos el modelo reentrenado al  set de prueba hacer la respectiva clasificación de cada una de las muestras. 

### Predicción del modelo

Se aplica el modelo reentrenado al dataset de prueba *test_dataset* y se evalúa la precisión del mismo por medio del accuracy, es decir, acá le pasamos mensajes sin etiquetas y le pedimos que prediga si son o no spam. El modelo para la tarea que fue entrenado presenta un accuracy de 1, es decir clasifica perfectamente el set de prueba. 

In [ ]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[ 2.7189324, -2.6125526],
       [-3.1758697,  3.172896 ],
       [ 3.001874 , -2.8654525],
       ...,
       [ 2.7466257, -2.626785 ],
       [-3.1808536,  3.1706238],
       [ 2.846122 , -2.7026985]], dtype=float32), label_ids=array([0, 1, 0, ..., 0, 1, 0], dtype=int32), metrics={'eval_loss': 0.02361285516193935})

In [ ]:
trainer.predict(test_dataset)[1].shape

(1115,)

#### Salidas

In [ ]:
output=trainer.predict(test_dataset)[1]
output

array([0, 1, 0, ..., 0, 1, 0], dtype=int32)

#### Matriz  de confusión, Accuracy

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

confusion_matrix=confusion_matrix(y_test,output)
confusion_matrix


array([[955,   0],
       [  0, 160]])

In [ ]:
acc=accuracy_score(y_test,output)
acc

1.0

#### Descarga del modelo

In [ ]:
trainer.save_model('tf_model')
